In [ ]:
# Loading all important Libraries
import numpy as np
import pickle
from keras.models import model_from_json
import pandas as pd
import datetime
import re
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')
class final:
    def load_files(self): # This function loads all important files
        #https://machinelearningmastery.com/save-load-keras-deep-learning-models/
        json_file = open('model.json', 'r') #Loads model
        loaded_model_json = json_file.read()
        json_file.close()
        self.model = model_from_json(loaded_model_json)
        self.model.load_weights("model.h5") # Loads weights of model

        with open('access_enc.pkl','rb') as file:
            self.access_enc=pickle.load(file) #Loads label encoder for access

        with open('lang_enc.pkl','rb') as file:
            self.lang_enc=pickle.load(file) #Loads label encoder for language

        with open('spider_enc.pkl','rb') as file:
            self.spider_enc=pickle.load(file) #Loads label encoder for spider
        self.new_data=pd.read_csv('final_data.csv')    #Loads data which is required to make prediction
    def find_access(self,page):
        #This function finds the client of the page for which we are making prediction
        k=max([i.start() for i in re.finditer('org_',page)])   #https://www.geeksforgeeks.org/python-all-occurrences-of-substring-in-string/
        if('all-access' in page[k:]):
            access='all_access'
        if('desktop' in page[k:]):
            access='desktop'
        if('mobile' in page[k:]): 
            access='mobile'
        k=access    
        access=self.access_enc.transform([access]).reshape(1,1)    
        return access,k 
    def find_lang(self,page): # This function finds language of the page for which we are making prediction
        index=page.find('.wikipedia')
        lang=page[index-1:index-3:-1][::-1]
        lang_dict={'de':'German','en':'English', 'es':'Spanish', 'fr':'French', 'ja':'Japanese', 'nt':'Media', 'ru':'Russian', 'zh':'Chinese'}
        language=lang_dict[lang]
        lang=self.lang_enc.transform([lang]).reshape(1,1)
        return lang,language
    def find_spider(self,page): #This page finds if the page was accessed by a spider or not.
        if('spider' in page):
            spider='spider'
        else:
            spider='non-spider' 
        k=spider  
        spider=self.spider_enc.transform([spider]).reshape(1,1)
        return spider,k
    def find_data(self,ind,date): 
        ''' This function returns the traffic on last 5 days on the page on which we are making prediction,
         this data is neccessary in order to make prediction and this data will be fed to loaded model.'''
        data=self.new_data.iloc[ind].values
        date1=datetime.date(2015,7,6)
        k=date.split('-')
        date2=datetime.date(int(k[0]),int(k[1]),int(k[2]))
        dif=(date2-date1).days
        data=np.log1p(data[dif+1:dif+6].astype(int))
        data=np.array(data).reshape(1,5,1)
        return data     
    def final_fun_1(self,ind,date): # This data takes index of the page and date as input for which we want to make prediction
        self.load_files()
        start=datetime.datetime.now()
        self.page=self.new_data['Page'].values[ind]
        access,access1=self.find_access(self.page)
        lang,language=self.find_lang(self.page)
        spider,spider1=self.find_spider(self.page)
        data=self.find_data(ind,date)
        predicted=int(np.round(np.expm1(self.model.predict([data,access,lang,spider])[0])[0]))
        
        x = PrettyTable()
        x = PrettyTable(["Client","Access", "Language",'Predicted','Time Taken'])
        row = [access1,spider1,language,predicted,datetime.datetime.now()-start]
        x.add_row(row)

        print(x)
        #Prints the client,access,language,time taken and predicted traffic on the page
    def final_fun_2(self,set): #This function takes index,date and actual traffic on the page as input, can also take a list of multiple inputs
        self.load_files()
        x = PrettyTable()
        x = PrettyTable(["Client","Access", "Language",'Predicted','Actual','SMAPE','Time Taken'])
        for val in set:
            start=datetime.datetime.now()
            self.page=self.new_data['Page'].values[val[0]]
            access,access1=self.find_access(self.page)
            lang,language=self.find_lang(self.page)
            spider,spider1=self.find_spider(self.page)
            data=self.find_data(val[0],val[1])
            actual=int(val[2])
            predicted=int(np.round(np.expm1(self.model.predict([data,access,lang,spider],steps=1)[0])[0]))
            smape=np.abs(actual-predicted)/((actual+predicted)/2)

            row = [access1,spider1,language,predicted,actual,np.round(smape,3),datetime.datetime.now()-start]
            x.add_row(row)
            #Print client,access,language,prediction,actual,time taken and SMAPE of the page
        print(x)



In [ ]:
test_object=final() #Making object of the class

In [ ]:
test_object.final_fun_1(6658,'2015-07-07') 

+--------+------------+----------+-----------+----------------+
| Client |   Access   | Language | Predicted |   Time Taken   |
+--------+------------+----------+-----------+----------------+
| mobile | non-spider | Spanish  |    362    | 0:00:00.443665 |
+--------+------------+----------+-----------+----------------+


In [ ]:
test_object.final_fun_1(893,'2016-04-24')

+------------+--------+----------+-----------+----------------+
|   Client   | Access | Language | Predicted |   Time Taken   |
+------------+--------+----------+-----------+----------------+
| all_access | spider |  German  |     96    | 0:00:00.364030 |
+------------+--------+----------+-----------+----------------+


In [ ]:
data=pd.read_csv('final_data.csv') # Reading data in oreder to provide actual data as input
test_object.final_fun_2([(8,'2015-09-07',data.at[8,'2015-09-07']),
                         (265,'2016-01-09',data.at[265,'2016-01-09']), #Providing a list of 4 inputs
                         (2187,'2016-04-23',data.at[2187,'2016-04-23']),
                         (18,'2015-10-07',data.at[18,'2015-10-07'])])

+------------+------------+----------+-----------+--------+-------+----------------+
|   Client   |   Access   | Language | Predicted | Actual | SMAPE |   Time Taken   |
+------------+------------+----------+-----------+--------+-------+----------------+
| all_access |   spider   | Chinese  |     4     |   5    | 0.222 | 0:00:00.435785 |
|  desktop   | non-spider |  French  |    115    |  109   | 0.054 | 0:00:00.040163 |
| all_access |   spider   | English  |    177    |  195   | 0.097 | 0:00:00.039685 |
| all_access |   spider   | Chinese  |    1178   |  1175  | 0.003 | 0:00:00.038544 |
+------------+------------+----------+-----------+--------+-------+----------------+
